In [1]:
from urllib.request import urlretrieve
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

spark = (
    SparkSession.builder.appName("project2 preprocessing")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.driver.memory", "4g")
    .config("spark.executor.memory", "8g")
    .getOrCreate()
)


file_path=f"../data/landing/income.csv"
sa2_code_path='../data/raw/sa2_code_name.csv'
output_path=f"../data/raw/test.csv"

df=spark.read.csv(file_path, header=True, inferSchema=True)
df_code=spark.read.csv(sa2_code_path, header=True, inferSchema=True)

df = df.withColumnRenamed('sa2_maincode_2016', 'SA2 code')

# Columns that should be included in every dataset
common_columns = ['FID', 'sa2_maincode_2016', 'geometry', 'sa2_name_2016', 'yr']
other_columns = "estmts_prsnl_incme_yr_endd_30_jne_emplye_ernrs_nm|estmts_prsnl_incme_yr_endd_30_jne_emplye_ernrs_mdn_age_yrs|estimates_personal_income_year_ended_30_june_total_employee_m|estimates_personal_income_year_ended_30_june_median_employee|estimates_personal_income_year_ended_30_june_mean_employee|estmts_prsnl_incme_yr_endd_30_jne_emplye_mn_srce_of_pc|estmts_prsnl_incme_yr_endd_30_jne_unncrprtd_bsnss_ernrs_nm|estm_prsnl_incme_yr_end_30_jne_uncrptd_bsn_erns_mdn_age_yrs|estmts_prsnl_incme_yr_endd_30_jne_ttl_unncrprtd_bsnss_m|estmts_prsnl_incme_yr_endd_30_jne_mdn_unncrprtd_bsnss|estmts_prsnl_incme_yr_endd_30_jne_mn_unncrprtd_bsnss|estmts_prsnl_incme_yr_endd_30_jne_unncrprtd_bsnss_mn_srce_of_pc|estmts_prsnl_incme_yr_endd_30_jne_invstmnt_ernrs_nm|estmts_prsnl_incme_yr_endd_30_jne_invstmnt_ernrs_mdn_age_yrs|estimates_personal_income_year_ended_30_june_total_investment_m|estimates_personal_income_year_ended_30_june_median_investment|estimates_personal_income_year_ended_30_june_mean_investment|estmts_prsnl_incme_yr_endd_30_jne_invstmnt_mn_srce_of_pc|estmts_prsnl_incme_yr_endd_30_jne_sprnntn_annty_ernrs_nm|estm_prsnl_incme_yr_end_30_jne_sprnt_anty_erns_mdn_age_yrs|estmts_prsnl_incme_yr_endd_30_jne_ttl_sprnntn_annty_m|estmts_prsnl_incme_yr_endd_30_jne_mdn_sprnntn_annty|estmts_prsnl_incme_yr_endd_30_jne_mn_sprnntn_annty|estmts_prsnl_incme_yr_endd_30_jne_sprnntn_annty_mn_srce_of_pc|estm_prsnl_incme_yr_end_30_jne_tl_erns_excl_gvrnmt_pns_alwncs_n|estm_prsnl_incme_yr_end_30_jne_tl_erns_excl_gvrnmt_pns_alwncs_m|estm_prsnl_incme_yr_end_30_jne_tl_excl_gvrnmt_pns_alwncs_m|estm_prsnl_incme_yr_end_30_jne_mdn_tl_excl_gvrnmt_pns_alwncs|estm_prsnl_incme_yr_end_30_jne_mn_tl_excl_gvrnmt_pns_alwncs|estm_prsnl_incme_yr_end_30_jne_tl_excl_gvrnmt_pns_alwncs_p802_r|estm_prsnl_incme_yr_end_30_jne_tl_excl_gvrnmt_pns_alwncs_p805_r|estm_prsnl_incme_yr_end_30_jne_tl_excl_gvrnmt_pns_alwncs_p205_r|estm_prsnl_incme_yr_end_30_jne_tl_excl_gvrnmt_pns_alwncs_p105_r|estm_prsnl_incme_yr_end_30_jne_tl_excl_gvrnmt_pns_alwncs_gni_cf|estm_prsnl_incme_yr_end_30_jne_tl_excl_gvrnmt_pns_alwncs_shre_o|estm_prsnl_incme_yr_end_30_jne_tl_excl_gvrnmt_pns_alwncs_shre_0|estm_prsnl_incme_yr_end_30_jne_tl_excl_gvrnmt_pns_alwncs_shre_1|estm_prsnl_incme_yr_end_30_jne_tl_excl_gvrnmt_pns_alwncs_lwst_q|estm_prsnl_incme_yr_end_30_jne_tl_excl_gvrnmt_pns_alwncs_scnd_q|estm_prsnl_incme_yr_end_30_jne_tl_excl_gvrnmt_pns_alwncs_thrd_q|estm_prsnl_incme_yr_end_30_jne_tl_excl_gvrnmt_pns_alwncs_hgst_q|grss_cptl_gns_rprtd_txpyrs_yr_endd_30_jne_by_prsns_nm|grss_cptl_gns_rprtd_txpyrs_yr_endd_30_jne_by_amnt_m|grss_cptl_gns_rprtd_txpyrs_yr_endd_30_jne_by_mdn|grss_cptl_gns_rprtd_txpyrs_yr_endd_30_jne_by_mn|slctd_gvrnmnt_pnsns_allwncs_30_jne_age_pnsn_cntrlnk_nm|slctd_gvrnmnt_pnsns_allwncs_30_jne_crr_pymnt_nm|slctd_gvrnmnt_pnsns_allwncs_30_jne_crr_allwnce_nm|slctd_gvrnmnt_pnsns_allwncs_30_jne_dsblty_spprt_pnsn_nm|slctd_gvrnmnt_pnsns_allwncs_30_jne_nwstrt_allwnce_nm|slctd_gvrnmnt_pnsns_allwncs_30_jne_prntng_pymnt_sngle_nm|slctd_gvrnmnt_pnsns_allwncs_30_jne_prntng_pymnt_prtnrd_nm|slctd_gvrnmt_pns_alwncs_30_jne_yth_alwnce_fl_tme_stdnprc_nm|slctd_gvrnmnt_pnsns_allwncs_30_jne_yth_allwnce_nm|slctd_gvrnmnt_pnsns_allwncs_30_jne_fmly_tx_bnft_nm|slctd_gvrnmnt_pnsns_allwncs_30_jne_fmly_tx_bnft_b_nm|slctd_gvrnmnt_pnsns_allwncs_30_jne_cmmnwlth_rnt_assstnce_nm|exprmntl_hshld_wlth_estmts_mdlld_prvte_dwllngs_mn_nt_wrth|ttl_prsnl_incme_wkly_prsns_agd_15_yrs_cnss_ernng_1_499_pr_wk_pc|tl_prsnl_incme_wkly_prsn_agd_15_yrs_cns_erng_500_999_pr_wk_pc|tl_prsnl_incme_wkly_prsn_agd_15_yrs_cns_erng_1000_1999_pr_wk_pc|tl_prsnl_incme_wkly_prsn_agd_15_yrs_cns_erng_2000_2999_pr_wk_pc|ttl_prsnl_incme_wkly_prsns_agd_15_yrs_cnss_ernng_3000_pr_wk_pc|ttl_prsnl_incme_wkly_prsns_agd_15_yrs_cnss_ernng_nl_pc|ttl_prsnl_incme_wkly_prsns_agd_15_yrs_cnss_ngtve_pc|tl_prsnl_incme_wkly_prsn_agd_15_yrs_cns_indqtly_dscrb_std_pc|equivalised_total_household_income_census_median_weekly".split("|")

df=df.join(df_code.select('SA2 code'), on='SA2 code', how='inner')


df_cleaned=df


your 131072x1 screen size is bogus. expect trouble
24/10/01 09:45:57 WARN Utils: Your hostname, Honor resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/10/01 09:45:57 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/01 09:45:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 42906)
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/local/lib/python3.11/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/usr/local/lib/python3.11/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/local/lib/python3.11/socketserver.py", line 755, in __init__
    self.handle()
  File "/home/david/project-2-group-real-estate-industry-project-19/project-2-group-real-estate-industry-project-19/.venv/lib/python3.11/site-packages/pyspark/accumulators.py", line 295, in handle
    poll(accum_updates)
  File "/home/david/project-2-group-real-estate-industry-project-19/project-2-group-real-estate-industry-project-19/.venv/lib/py

In [2]:
columns_with_null=[]

for column in df_cleaned.columns:
    # Filter rows where a column has null values
    df_null = df_cleaned.filter(df_cleaned[column].isNull())

# Count rows with null values
    null_count = df_null.count()

    if (null_count>0):
        columns_with_null.append(column)


In [3]:
from pyspark.sql.functions import col, when, avg
from pyspark.sql import Window
from pyspark.sql.functions import coalesce, lag, lead


def null_filler(df, column_name):
# Create a window specification partitioned by sa2_maincode_2016 and ordered by yr
    window_spec = Window.partitionBy('SA2 code').orderBy('yr')

# Use lag (previous year) and lead (next year) functions
    df_with_adjacent_values = df.withColumn('prev_year_value', lag(column_name).over(window_spec)) \
                            .withColumn('next_year_value', lead(column_name).over(window_spec))


# Calculate the average of prev_year_value and next_year_value, ignoring nulls
    df_filled = df_with_adjacent_values.withColumn('filled_value',
        when(
            col(column_name).isNull(), 
            (coalesce(col('prev_year_value'), col('next_year_value')) + coalesce(col('next_year_value'), col('prev_year_value'))) / 2
        ).otherwise(col(column_name))
    )

# Handle cases where both prev_year_value and next_year_value are null
    df_filled = df_filled.withColumn('filled_value',
        when(
            col('filled_value').isNull(), 
            lag('prev_year_value', 2).over(window_spec)  # or lead for next 2 years
        ).otherwise(col('filled_value'))
    )

    df_final = df_filled.withColumn(column_name, col('filled_value')).drop('prev_year_value', 'next_year_value', 'filled_value')
    return(df_final)


In [4]:
def null_filler_multi(df, columns_list):
    for column in columns_list:
        df=null_filler(df, column)
    return df

df_filled=null_filler_multi(df_cleaned, columns_with_null)

In [6]:
columns_wanted=['SA2 code', 'sa2_name_2016', 'yr', 'equivalised_total_household_income_census_median_weekly']
output_path='../data/raw/income_filled.csv'

df=df_filled.select(columns_wanted)
df.write.csv(output_path, header=True, mode="overwrite")

24/10/01 09:51:05 WARN BaseSessionStateBuilder$$anon$2: Max iterations (100) reached for batch Operator Optimization before Inferring Filters, please set 'spark.sql.optimizer.maxIterations' to a larger value.


In [1]:
from urllib.request import urlretrieve
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

spark = (
    SparkSession.builder.appName("project2 preprocessing")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.driver.memory", "4g")
    .config("spark.executor.memory", "8g")
    .getOrCreate()
)


file_path=f"../data/raw/income_filled.csv"

df=spark.read.csv(file_path, header=True, inferSchema=True)

your 131072x1 screen size is bogus. expect trouble
24/10/01 11:41:48 WARN Utils: Your hostname, Honor resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/10/01 11:41:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/01 11:41:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/10/01 11:41:49 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/10/01 11:41:49 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [18]:

df.filter(F.col('SA2 code')==203021487).show()

+--------+-------------+---+-------------------------------------------------------+
|SA2 code|sa2_name_2016| yr|equivalised_total_household_income_census_median_weekly|
+--------+-------------+---+-------------------------------------------------------+
+--------+-------------+---+-------------------------------------------------------+



In [16]:
from pyspark.sql import Window

window_spec = Window.partitionBy("SA2 code").orderBy(F.desc("yr"))

# Step 2: Use the Window function to get the last year value
df_last_year_value = (
    df.withColumn("last_year", F.max("yr").over(window_spec))  # Get the last year per SA2 code
    .filter(F.col("yr") == F.col("last_year"))  # Keep only rows where the year is the last year
)

# Step 3: Select the desired columns
df_last_year_value = df_last_year_value.select(
    "SA2 code", 
    "sa2_name_2016", 
    "equivalised_total_household_income_census_median_weekly"
)


output_path=f"../data/raw/income_only.csv"

df_last_year_value.write.csv(output_path, header=True, mode="overwrite")

In [32]:
idlist=[208031192,205051099,206041120,205031092,206041127,210011227, 205021080]

df.filter(col('SA2 code').isin(idlist))

SA2 code,sa2_name_2016,yr,equivalised_total_household_income_census_median_weekly
205031092,Wilsons Promontory,2016,1375.0
205031092,Wilsons Promontory,2017,1375.0
205051099,Alps - West,2011,350.0
205051099,Alps - West,2014,350.0
205051099,Alps - West,2015,575.0
205051099,Alps - West,2016,575.0
205051099,Alps - West,2017,575.0
206041120,Flemington Raceco...,2011,1218.0
206041120,Flemington Raceco...,2014,1218.0
206041120,Flemington Raceco...,2015,1512.0


In [36]:
from pyspark.sql import functions as F

# Assuming the original DataFrame 'df' is already created

# Group by 'id' and aggregate to check if there's any row with year 2019
df_no_2019 = df.groupBy("SA2 code").agg(
    F.max(F.when(col("yr") == 2019, 1).otherwise(0)).alias("has_2019")
)

# Filter for IDs that do not have year 2019
df_no_2019_filtered = df_no_2019.filter(col("has_2019") == 0)

# Show the result
df_no_2019_filtered.show()

df_joined = df_no_2019_filtered.join(df, on='SA2 code')


+---------+--------+
| SA2 code|has_2019|
+---------+--------+
|208031192|       0|
|205021080|       0|
|205051099|       0|
|206041120|       0|
|205031092|       0|
|206041127|       0|
|210011227|       0|
+---------+--------+



In [37]:
df_2019_nulls = df.filter(col('yr') == 2019)
df_2019_nulls=df_2019_nulls.filter(col('equivalised_total_household_income_census_median_weekly').isNull())
df_2019_nulls.show()

+---------+-----------------+----+-------------------------------------------------------+
| SA2 code|    sa2_name_2016|  yr|equivalised_total_household_income_census_median_weekly|
+---------+-----------------+----+-------------------------------------------------------+
|210051248|Melbourne Airport|2019|                                                   NULL|
+---------+-----------------+----+-------------------------------------------------------+



In [38]:
df_2011 = df.filter(col('yr') == 2011).select('SA2 code', col('equivalised_total_household_income_census_median_weekly').alias('value_2011'))
df_2019 = df.filter(col('yr') == 2019).select('SA2 code', col('equivalised_total_household_income_census_median_weekly').alias('value_2019'))

# Join the two DataFrames on 'id'
df_joined = df_2011.join(df_2019, on='SA2 code')

# Calculate the CAGR
df_with_cagr = df_joined.withColumn(
    'CAGR', 
    (pow(col('value_2019') / col('value_2011'), 1 / 8) - 1)
)

df_with_cagr=df_with_cagr.join(df.select('SA2 code', 'sa2_name_2016').dropDuplicates(["SA2 code"]), on='SA2 code')

# Show the result
df_with_cagr.show()

+---------+----------+----------+--------------------+--------------------+
| SA2 code|value_2011|value_2019|                CAGR|       sa2_name_2016|
+---------+----------+----------+--------------------+--------------------+
|201011001|     786.0|     914.0|0.019038180747419586|           Alfredton|
|201011002|     762.0|     903.0| 0.02144878781001558|            Ballarat|
|201011005|     812.0|     920.0|0.015731625603674804|           Buninyong|
|201011006|     643.0|     761.0|0.021284428971496627|           Delacombe|
|201011007|     736.0|     867.0|0.020687180934804372|       Smythes Creek|
|201011008|     565.0|     635.0|0.014707007710210496|Wendouree - Miner...|
|201021009|     697.0|     837.0| 0.02314358351846635|Bacchus Marsh Region|
|201021010|     568.0|     643.0|0.015623706758379097|   Creswick - Clunes|
|201021011|     576.0|     689.0|0.022644277110845712|          Daylesford|
|201021012|     691.0|     782.0|0.015584556648718495|       Gordon (Vic.)|
|201031013| 

In [40]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder.appName("project2 preprocessing")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.driver.memory", "4g")
    .config("spark.executor.memory", "8g")
    .getOrCreate()
)

df_wanted=df_with_cagr.withColumnRenamed('CAGR', 'avg_yearly_growth_rate')
columns_wanted=['SA2 code', 'sa2_name_2016', 'value_2019', "avg_yearly_growth_rate"]

output_path=f"../data/raw/income.csv"
df_wanted=df_wanted.select(columns_wanted)
df_wanted.write.csv(output_path, header=True, mode="overwrite")

In [44]:
df_full=spark.read.csv('../data/raw/population.csv', header=True)

# Extract distinct ids from the DataFrame
distinct_ids_df = df_full.withColumnRenamed("SA2 code", 'id').select('id').distinct()

# Collect the distinct ids into a Python list
existing_ids = distinct_ids_df.rdd.map(lambda row: row.id).collect()

# Convert the existing ids to a set
id_set = set(existing_ids)

# Extract distinct ids from the DataFrame
distinct_ids_df = df_wanted.withColumnRenamed("SA2 code", 'id').select("id").distinct()

# Collect the distinct ids into a Python list
existing_ids = distinct_ids_df.rdd.map(lambda row: row.id).collect()

# Convert the existing ids to a set
existing_ids_set = set(existing_ids)

# Find the ids that are in id_list but not in the DataFrame
missing_ids = id_set - existing_ids_set

# Show the missing ids
print(f"IDs missing from DataFrame: {len(missing_ids)}")


IDs missing from DataFrame: 522


In [17]:
spark = (
    SparkSession.builder.appName("project2 preprocessing")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.driver.memory", "4g")
    .config("spark.executor.memory", "8g")
    .getOrCreate()
)

df=spark.read.csv(output_path)
df.show()

+---------+--------------------+----------+--------------------+
|      _c0|                 _c1|       _c2|                 _c3|
+---------+--------------------+----------+--------------------+
| SA2 code|       sa2_name_2016|value_2019|avg_yearly_growth...|
|201011001|           Alfredton|     914.0|0.019038180747419586|
|201011002|            Ballarat|     903.0| 0.02144878781001558|
|201011005|           Buninyong|     920.0|0.015731625603674804|
|201011006|           Delacombe|     761.0|0.021284428971496627|
|201011007|       Smythes Creek|     867.0|0.020687180934804372|
|201011008|Wendouree - Miner...|     635.0|0.014707007710210496|
|201021009|Bacchus Marsh Region|     837.0| 0.02314358351846635|
|201021010|   Creswick - Clunes|     643.0|0.015623706758379097|
|201021011|          Daylesford|     689.0|0.022644277110845712|
|201021012|       Gordon (Vic.)|     782.0|0.015584556648718495|
|201031013|               Avoca|     535.0| 0.01933664440418581|
|201031014|            Be